# Intern Interview Technical task
Task: Get data from coingecko (cryptocurrency data provider https://www.coingecko.com/en) and store ECR-20 token market data into a SQL database

## Step 0: Install pycoingecko API and import libaries

In [1]:
### install pycoingecko API
! pip install pycoingecko

In [3]:
### import libraries

# Data Processing
import pandas as pd

# SQL related
import sqlite3
conn = sqlite3.connect('crypto_data.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - address
c.execute('''CREATE TABLE address
             ([token_id] Text PRIMARY KEY,[contract_address] text)''')
conn.commit()

# Create table - market
c.execute('''CREATE TABLE market
             ([token_id] Text PRIMARY KEY,[date] date, [time] time, [price] float, [volume] float)''')
conn.commit()

# reference: https://datatofish.com/create-database-python-using-sqlite3/

# Time
from datetime import datetime, timedelta
import time

# API
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

## Step 1: Check the API status

In [4]:
# check the API status
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

## Step 2: Write a helper function to convert timestamp to datetime

In [5]:
# helper function
# Enter the time in seconds (with 10 digits) or milliseconds (with 13 digits),
# and convert the time to the format of YYYY-MM-DD hh:mm:ss 
def convert_timestamp(t):
    if t / 1e12 > 1: # 13 digits
        time_stamp = float(t/1000)
        formatted_time = datetime.fromtimestamp(time_stamp)
        return formatted_time
    else: # 10 digits
        return datetime.fromtimestamp(t)
# references: https://www.programmersought.com/article/34094898312/

## Step 3-1: Store the address information for the ERC-20 coins ranked top 80 by market cap

In [6]:
# create a dictionary to store the (token_id, address) pair
top80_id_address_dict = {}

# the list for the top 80 Ethereum based coins
top80_coin_list = ['tether', 'usd-coin', 'uniswap', 'chainlink', 'matic-network', 'binance-usd', 'wrapped-bitcoin',
              'aave', 'shiba-inu', 'dai', 'okb', 'compound-ether', 'maker', 'crypto-com-chain', 'compound-usd-coin',
              'ftx-token', 'cdai', 'celsius-degree-token', 'huobi-token', 'compound-governance-token', 'havven',
              'sushi', 'telcoin', 'terrausd', 'leo-token', 'holotoken', 'yearn-finance', 'waves', 'enjincoin',
              'amp-token', 'basic-attention-token', 'paxos-standard', 'true-usd', 'nexo', 'huobi-btc']

for token in top80_coin_list:
    top80_id_address_dict[token] = cg.get_coin_by_id(id = token, vs_currencies='usd')["contract_address"]
    time.sleep(0.5) # prevent exceeding the request limit per second

## Step 3-2: Convert the dictionary to the pandas DataFrame

In [7]:
address_df = pd.DataFrame.from_dict(top80_id_address_dict, orient='index', columns = ["contract_address"])
address_df["token_id"] = address_df.index
address_df.reset_index(drop = True, inplace = True)
address_df = address_df[["token_id", "contract_address"]]

# store the dataframe to SQL database
address_df.to_sql('address', conn, if_exists = 'replace', index = False)
address_df

,token_id,contract_address
0,tether,0xdac17f958d2ee523a2206206994597c13d831ec7
1,usd-coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
2,uniswap,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
3,chainlink,0x514910771af9ca656af840dff83e8264ecf986ca
4,matic-network,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
5,binance-usd,0x4fabb145d64652a948d72533023f6e7a623c7c53
6,wrapped-bitcoin,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
7,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
8,shiba-inu,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce
9,dai,0x6b175474e89094c44da98b954eedeac495271d0f


## Step 4 Get the price and volume data for the ERC-20 coins ranked top 80 and store it to a SQL database

In [8]:
# create an empty dataframe with 5 columns
price_volume_df = pd.DataFrame(columns = ["token_id", "date", "time", "price", "volume"])

for key in top80_id_address_dict.keys(): # for each coin
    # store the price and volume history for each coin
    prices = cg.get_coin_market_chart_by_id(id = key, vs_currency = "usd", days = 90)["prices"]
    volumes = cg.get_coin_market_chart_by_id(id = key, vs_currency = "usd", days = 90)["total_volumes"]

    price_df = pd.DataFrame(prices, columns = ["full_time", "price"])
    price_df.full_time = price_df.full_time.apply(convert_timestamp)

    volume_df = pd.DataFrame(volumes, columns = ["full_time", "volume"])
    volume_df.full_time = volume_df.full_time.apply(convert_timestamp)

    # combine two dataframes 
    df = price_df.join(volume_df.set_index('full_time'), on='full_time')
    
    # add a column called token_id and the values are the coin symbol
    df["token_id"] = key

    # convert the full time to two columns: date and time
    df["date"] = df.full_time.dt.date
    df["time"] = df.full_time.dt.time

    df = df[["token_id", "date", "time", "price", "volume"]].dropna()

    # concatenate the history for all coins
    price_volume_df = pd.concat([price_volume_df, df], ignore_index=True)
    
    # prevent exceeding the request limit per second
    time.sleep(2)

# store the dataframe to SQL database 
price_volume_df.to_sql('market', conn, if_exists = 'replace', index = False)
price_volume_df

,token_id,date,time,price,volume
0,tether,2021-03-02,17:07:10.162000,1.000637,1.043994e+11
1,tether,2021-03-02,18:08:46.828000,1.002515,1.034332e+11
2,tether,2021-03-02,19:06:08.882000,1.002278,1.033254e+11
3,tether,2021-03-02,20:04:55.370000,0.999977,1.004860e+11
4,tether,2021-03-02,21:22:55.318000,1.004368,9.870982e+10
...,...,...,...,...,...
75572,huobi-btc,2021-05-31,12:34:20.535000,34649.712386,4.372830e+06
75573,huobi-btc,2021-05-31,13:34:25.207000,34518.868205,4.356124e+06
75574,huobi-btc,2021-05-31,14:51:38.501000,34887.890527,4.402547e+06
75575,huobi-btc,2021-05-31,15:45:34.073000,35704.433629,6.970934e+05


## Step 5: Close the connection

In [9]:
c.close()
conn.close()